In [30]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble, Aer
from qiskit.visualization import plot_histogram
from math import pi

def circuitMaker(n_qubits, n_cbits, encodings, parameters):
    # create quantum circuit
    qr = QuantumRegister(n_qubits, 'qubit')
    cr = ClassicalRegister(n_cbits, 'cr')
    qc = QuantumCircuit()
    qc.add_register(qr)
    qc.add_register(cr)
    
    # encoding block
    encoder0, encoder1 = encoder(encodings)
    # print(encodings, encoder1)
    # qc.rx(pi, encoder0)
    # qc.rz(pi, encoder0)
    qc.rx(pi, encoder1)
    qc.rz(pi, encoder1)
    qc.barrier(qr)
    
    for i in range(n_qubits):
        # entangle block
        if i < n_qubits - 1:
            qc.cnot(i, i+1)
        else:
            qc.barrier(qr)

    for i in range(n_qubits):
        # variational block
        qc.u3(parameters[i][0], parameters[i][1], parameters[i][2], i)

    # measurement
    qc.measure(qr, cr)
    
    return qc

def encoder(encodings):
    return [i for i, b in enumerate(encodings) if b == '0'], [i for i, b in enumerate(encodings) if b == '1']


In [32]:
parameters = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]]
circuit_size = 4
b_state = '1011'

qc = circuitMaker(circuit_size, circuit_size, b_state, parameters)
qc.draw()

┌───────┐┌───────┐ ░                 ░ ┌───────────┐
qubit_0: ┤ Rx(π) ├┤ Rz(π) ├─░───■─────────────░─┤ U3(1,1,1) ├
         └───────┘└───────┘ ░ ┌─┴─┐           ░ ├───────────┤
qubit_1: ───────────────────░─┤ X ├──■────────░─┤ U3(1,1,1) ├
         ┌───────┐┌───────┐ ░ └───┘┌─┴─┐      ░ ├───────────┤
qubit_2: ┤ Rx(π) ├┤ Rz(π) ├─░──────┤ X ├──■───░─┤ U3(1,1,1) ├
         ├───────┤├───────┤ ░      └───┘┌─┴─┐ ░ ├───────────┤
qubit_3: ┤ Rx(π) ├┤ Rz(π) ├─░───────────┤ X ├─░─┤ U3(1,1,1) ├
         └───────┘└───────┘ ░           └───┘ ░ └───────────┘
   cr: 4/════════════════════════════════════════════════════